In [12]:
!pip install librosa

In [318]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Input, Dropout

import os
import librosa
import librosa.display
import pickle

from tqdm.notebook import tqdm

In [179]:
emotions = { '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad', '05': 'angry', 
            '06': 'fearful', '07': 'disgust', '08': 'surprised'}

def get_label(filename):
    '''This function takes the audio file name as a parameter, e.g. 03-01-01-01-01-01-01.wav and returns
    the label as a text, e.g. neutral'''
    tags = filename.split('-')
    emotion = emotions[tags[2]]
    return emotion

In [180]:
# create a dataframe containing the files paths with their labels

files_paths = {'filepath': [], 'label': []}
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        
        files_paths['filepath'].append(filepath)
        files_paths['label'].append(get_label(filename))
        

files_paths = pd.DataFrame(files_paths)

In [181]:
files_paths

,filepath,label
0,data\Actor_01\03-01-01-01-01-01-01.wav,neutral
1,data\Actor_01\03-01-01-01-01-02-01.wav,neutral
2,data\Actor_01\03-01-01-01-02-01-01.wav,neutral
3,data\Actor_01\03-01-01-01-02-02-01.wav,neutral
4,data\Actor_01\03-01-02-01-01-01-01.wav,calm
...,...,...
1435,data\Actor_24\03-01-08-01-02-02-24.wav,surprised
1436,data\Actor_24\03-01-08-02-01-01-24.wav,surprised
1437,data\Actor_24\03-01-08-02-01-02-24.wav,surprised
1438,data\Actor_24\03-01-08-02-02-01-24.wav,surprised


In [134]:
# create a dataframe with the features (mfccs) calculated

df = pd.DataFrame(columns=['feature'])

# The following block is commented out because the results of it are saved in a file 

# i = 0
# for filepath in tqdm(files_paths.filepath):
#     audio, sample_rate = librosa.load(filepath)
    
#     sample_rate = np.array(sample_rate)
#     mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
#     mfccs_mean = np.mean(mfccs, axis=0)
#     df.loc[i] = [mfccs_mean]
    
#     i += 1

In [136]:
# saving the results of the last block in a file
# df.to_pickle('RAVDESS-mfccs')

In [311]:
# load the saved results from the file
df = pd.read_pickle('RAVDESS-mfccs')
df.head()

,feature
0,"[-21.432735, -21.432735, -21.432735, -21.43273..."
1,"[-21.622257, -21.622257, -21.622257, -21.62225..."
2,"[-18.101141, -17.714779, -16.729706, -17.67694..."
3,"[-19.077229, -19.447943, -19.401802, -18.92272..."
4,"[-20.582228, -19.17782, -18.60321, -19.964561,..."


In [312]:
# create one big dataframe containing the file paths, their labels and their features
df = pd.concat([files_paths, pd.DataFrame(df['feature'].values.tolist())], axis=1)

In [313]:
df

,filepath,label,0,1,2,3,4,5,6,7,...,218,219,220,221,222,223,224,225,226,227
0,data\Actor_01\03-01-01-01-01-01-01.wav,neutral,-21.432735,-21.432735,-21.432735,-21.432735,-21.432735,-21.432735,-21.432735,-21.277515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,data\Actor_01\03-01-01-01-01-02-01.wav,neutral,-21.622257,-21.622257,-21.622257,-21.622257,-20.095755,-19.467009,-20.704380,-20.392015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,data\Actor_01\03-01-01-01-02-01-01.wav,neutral,-18.101141,-17.714779,-16.729706,-17.676943,-18.284256,-17.920685,-18.152332,-17.445875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,data\Actor_01\03-01-01-01-02-02-01.wav,neutral,-19.077229,-19.447943,-19.401802,-18.922726,-18.963850,-18.131964,-19.041887,-18.886808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,data\Actor_01\03-01-02-01-01-01-01.wav,calm,-20.582228,-19.177820,-18.603210,-19.964561,-20.390726,-20.453844,-21.788517,-21.550198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,data\Actor_24\03-01-08-01-02-02-24.wav,surprised,-19.122814,-19.122814,-19.122814,-19.122814,-19.122814,-19.122814,-19.122814,-19.122814,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1436,data\Actor_24\03-01-08-02-01-01-24.wav,surprised,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1437,data\Actor_24\03-01-08-02-01-02-24.wav,surprised,-18.535404,-18.535404,-18.535404,-18.535404,-18.535404,-18.535404,-18.535404,-18.535404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1438,data\Actor_24\03-01-08-02-02-01-24.wav,surprised,-17.836590,-17.836590,-17.836590,-17.836590,-17.836590,-17.824371,-17.803312,-17.827566,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [314]:
# replace NaN by 0
df = df.fillna(0)
df

,filepath,label,0,1,2,3,4,5,6,7,...,218,219,220,221,222,223,224,225,226,227
0,data\Actor_01\03-01-01-01-01-01-01.wav,neutral,-21.432735,-21.432735,-21.432735,-21.432735,-21.432735,-21.432735,-21.432735,-21.277515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,data\Actor_01\03-01-01-01-01-02-01.wav,neutral,-21.622257,-21.622257,-21.622257,-21.622257,-20.095755,-19.467009,-20.704380,-20.392015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,data\Actor_01\03-01-01-01-02-01-01.wav,neutral,-18.101141,-17.714779,-16.729706,-17.676943,-18.284256,-17.920685,-18.152332,-17.445875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,data\Actor_01\03-01-01-01-02-02-01.wav,neutral,-19.077229,-19.447943,-19.401802,-18.922726,-18.963850,-18.131964,-19.041887,-18.886808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,data\Actor_01\03-01-02-01-01-01-01.wav,calm,-20.582228,-19.177820,-18.603210,-19.964561,-20.390726,-20.453844,-21.788517,-21.550198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,data\Actor_24\03-01-08-01-02-02-24.wav,surprised,-19.122814,-19.122814,-19.122814,-19.122814,-19.122814,-19.122814,-19.122814,-19.122814,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1436,data\Actor_24\03-01-08-02-01-01-24.wav,surprised,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1437,data\Actor_24\03-01-08-02-01-02-24.wav,surprised,-18.535404,-18.535404,-18.535404,-18.535404,-18.535404,-18.535404,-18.535404,-18.535404,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1438,data\Actor_24\03-01-08-02-02-01-24.wav,surprised,-17.836590,-17.836590,-17.836590,-17.836590,-17.836590,-17.824371,-17.803312,-17.827566,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [315]:
# split the data: 80% train and 20% validation
X_train, X_test, y_train, y_test = train_test_split(df.drop(['filepath', 'label'], axis=1),
                                                   df['label'],
                                                   test_size=0.2,
                                                   shuffle=True,
                                                   random_state=42)

X_train[:10]

,0,1,2,3,4,5,6,7,8,9,...,218,219,220,221,222,223,224,225,226,227
1148,-25.653189,-25.653189,-25.653189,-25.653189,-25.653189,-25.653189,-25.653189,-25.653189,-25.653189,-25.653189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
677,-16.186077,-16.186077,-16.186077,-16.186077,-16.186077,-16.186077,-16.186077,-16.186077,-16.186077,-16.186077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
221,-15.161586,-15.161586,-15.161586,-15.161586,-15.161586,-15.161586,-15.161586,-15.161586,-15.161586,-15.161586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113,-19.642263,-19.642263,-19.642263,-19.642263,-19.642263,-19.642263,-19.642263,-19.642263,-19.642263,-19.642263,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
845,-13.947101,-14.531044,-15.090982,-14.969928,-14.906607,-15.177971,-14.709198,-14.394396,-14.194595,-14.864087,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1293,-16.140579,-16.444485,-16.444485,-16.302029,-16.444485,-16.444485,-16.444485,-15.763693,-15.243236,-16.430103,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
174,-19.254173,-19.254173,-19.254173,-19.254173,-19.254173,-19.254173,-19.254173,-19.254173,-19.254173,-19.254173,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
479,-14.281309,-14.782087,-15.179446,-14.877653,-15.576762,-15.649673,-14.376600,-13.651395,-13.175242,-13.773949,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
346,-24.008503,-24.008503,-24.008503,-24.008503,-24.008503,-24.008503,-23.468340,-22.172907,-22.580055,-23.948896,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1436,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,-18.309284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [316]:
# Normalizing the values
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

X_train[:10]

,0,1,2,3,4,5,6,7,8,9,...,218,219,220,221,222,223,224,225,226,227
1148,-2.606499,-2.647716,-2.684357,-2.695999,-2.708532,-2.711116,-2.733128,-2.731972,-2.740366,-2.735461,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476
677,0.452697,0.404067,0.400542,0.392150,0.388615,0.386758,0.372390,0.373810,0.373541,0.364191,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476
221,0.783751,0.734318,0.734376,0.726337,0.723775,0.721997,0.708456,0.709905,0.710515,0.699622,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476
113,-0.664133,-0.710056,-0.725671,-0.735250,-0.742070,-0.744192,-0.761351,-0.760027,-0.763262,-0.767408,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476
845,1.176199,0.937577,0.757383,0.788855,0.807191,0.716636,0.856854,0.961589,1.028576,0.797027,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476
1293,0.467399,0.320768,0.316338,0.354327,0.304078,0.302201,0.287624,0.512378,0.683659,0.284294,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476
174,-0.538725,-0.584953,-0.599211,-0.608656,-0.615107,-0.617199,-0.634045,-0.632710,-0.635612,-0.640343,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476
479,1.068203,0.856652,0.728556,0.818955,0.587951,0.562283,0.965957,1.205338,1.363860,1.153952,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476
346,-2.075037,-2.117541,-2.148429,-2.159506,-2.170476,-2.172934,-2.016427,-1.590230,-1.729556,-2.177454,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476
1436,-0.233395,-0.280362,-0.291314,-0.300435,-0.305988,-0.308008,-0.324091,-0.322730,-0.324821,-0.330974,...,0.041458,0.041499,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476,0.029476


In [254]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [266]:
# mapping the labels into numbers: angry -> 0, calm -> 1, etc.
encoder = LabelEncoder()
encoder.fit(y_train)

# encoding the mapped values into vectors (one hot encoding), e.g.:
# 0 -> [1, 0, 0, 0, 0, 0, 0, 0]
# 1 -> [0, 1, 0, 0, 0, 0, 0, 0]
# 5 -> [0, 0, 0, 0, 0, 1, 0, 0]
y_train_encoded = tf.keras.utils.to_categorical(encoder.transform(y_train))
y_test_encoded = tf.keras.utils.to_categorical(encoder.transform(y_test))

print(encoder.classes_)
print(y_test)
print(y_test_encoded)

['angry' 'calm' 'disgust' 'fearful' 'happy' 'neutral' 'sad' 'surprised']
['disgust' 'calm' 'calm' 'calm' 'angry' 'calm' 'calm' 'fearful'
 'surprised' 'calm' 'disgust' 'surprised' 'sad' 'fearful' 'sad' 'disgust'
 'angry' 'fearful' 'calm' 'disgust' 'surprised' 'neutral' 'neutral' 'sad'
 'fearful' 'sad' 'surprised' 'surprised' 'disgust' 'angry' 'surprised'
 'angry' 'happy' 'fearful' 'surprised' 'calm' 'disgust' 'angry' 'happy'
 'angry' 'calm' 'neutral' 'calm' 'angry' 'surprised' 'calm' 'calm'
 'neutral' 'fearful' 'sad' 'sad' 'surprised' 'surprised' 'angry' 'neutral'
 'calm' 'calm' 'surprised' 'happy' 'neutral' 'surprised' 'fearful'
 'surprised' 'surprised' 'fearful' 'happy' 'calm' 'disgust' 'disgust'
 'sad' 'happy' 'surprised' 'calm' 'happy' 'surprised' 'calm' 'angry'
 'surprised' 'sad' 'disgust' 'happy' 'happy' 'surprised' 'calm' 'happy'
 'angry' 'fearful' 'calm' 'happy' 'calm' 'neutral' 'calm' 'angry' 'angry'
 'neutral' 'disgust' 'surprised' 'happy' 'happy' 'surprised' 'disgust'
 'fearf

In [361]:
# # creating the neural network model
# model = tf.keras.Sequential()

# # specifying the size of the input
# model.add(Input(X_train.shape[1]))

# model.add(Dense(units=128, activation ='relu', kernel_regularizer=tf.keras.regularizers.L1(0.02)))
# model.add(Dense(units=64, activation ='relu', kernel_regularizer=tf.keras.regularizers.L1(0.02)))
# model.add(Dense(units=8, activation ='softmax'))

model = tf.keras.Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(8)) # Target class number
model.add(Activation('softmax'))

X_train__ = np.expand_dims(X_train, axis=2)
X_test__ = np.expand_dims(X_test, axis=2)

# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
# opt = tf.keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)

model.summary()

Model: "sequential_59"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_50 (Conv1D)           (None, 228, 256)          2304      
_________________________________________________________________
activation_99 (Activation)   (None, 228, 256)          0         
_________________________________________________________________
conv1d_51 (Conv1D)           (None, 228, 256)          524544    
_________________________________________________________________
batch_normalization_13 (Batc (None, 228, 256)          1024      
_________________________________________________________________
activation_100 (Activation)  (None, 228, 256)          0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 228, 256)          0         
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 28, 256)         

In [362]:
opt = tf.keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
history = model.fit(X_train__, y_train_encoded, batch_size=16, epochs=50, validation_data=(X_test__, y_test_encoded))

Epoch 1/50
72/72 [==============================] - 16s 227ms/step - loss: 1.0381 - accuracy: 0.6727 - val_loss: 1.5780 - val_accuracy: 0.4375
Epoch 2/50
72/72 [==============================] - 16s 228ms/step - loss: 1.0179 - accuracy: 0.6797 - val_loss: 1.5589 - val_accuracy: 0.4306
Epoch 3/50
72/72 [==============================] - 16s 229ms/step - loss: 1.0147 - accuracy: 0.6910 - val_loss: 1.5775 - val_accuracy: 0.4167
Epoch 4/50
72/72 [==============================] - 16s 229ms/step - loss: 0.9980 - accuracy: 0.6979 - val_loss: 1.5564 - val_accuracy: 0.4375
Epoch 5/50
72/72 [==============================] - 16s 228ms/step - loss: 0.9946 - accuracy: 0.6997 - val_loss: 1.5615 - val_accuracy: 0.4479
Epoch 6/50
72/72 [==============================] - 17s 230ms/step - loss: 0.9790 - accuracy: 0.7161 - val_loss: 1.5857 - val_accuracy: 0.4028
Epoch 7/50
72/72 [==============================] - 16s 229ms/step - loss: 0.9642 - accuracy: 0.7118 - val_loss: 1.5638 - val_accuracy: 0.4271